In [1]:
import numpy as np
import os
from netCDF4 import Dataset

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
import os, sys
module_path = os.path.abspath(os.path.join('C:/Users/Adam.000/Dropbox/Code/DisCo/'))
sys.path.append(module_path)
from visuals import *

In [3]:
matplotlib.use("TkAgg")
%matplotlib inline

In [4]:
datadir = "D:/Adam/Extracted-Disco-Data/CAM5-1-0.25degree_All-Hist_est1_v3_run1.cam.h2.2015-08-29-00000.nc"
obs_load = Dataset(datadir, 'r')
lats = obs_load['lat'][:]
lons = obs_load['lon'][:]

In [5]:
lats[0]

-90.0

In [6]:
lats[99]

-66.76662320730117

In [7]:
antarctic_lat = 99
stropic_lat = 284
ntropic_lat = 483
arctic_lat = 668

## load fields

In [8]:
run_dir =  "D:/Adam/Extracted-Disco-Data/AR-Extracted-Data/netcdf_data/"
obs_fields = []
lcsdir =  "D:/Adam/Extracted-Disco-Data/IVT_alt/result-16/fields/"
lcsfiles = sorted(os.listdir(lcsdir))

In [9]:
for (i,s) in enumerate(lcsfiles):
    index = i%8
    if index == 0:
        obs_name = s[:-6]+'00000.nc'
        # obs_name = obs_name[:43]+obs_name[44:46]+'2'+obs_name[47:]
        obs_load = Dataset(run_dir+obs_name, 'r')["PRECT"][:]
    obs_field = obs_load[index]
    obs_fields.append(obs_field)

In [10]:
precip_field = np.stack(obs_fields)

In [11]:
np.shape(precip_field)

(500, 768, 1152)

In [12]:
lcs_fields = []

for i,s in enumerate(lcsfiles):
    s_field = np.load(lcsdir+s)
    lcs_fields.append(s_field)
state_field = np.vstack(lcs_fields)

In [13]:
np.shape(state_field)

(500, 768, 1152)

## create masked array to mask out background state from the LCS field

In [14]:
filteredIVT = np.ones(np.shape(state_field), dtype=int)
filteredIVT[state_field==13] = 0
filteredIVT[state_field==0] = 0 # also include boundary "null" state

In [15]:
maskedIVT = np.ma.masked_array(filteredIVT, filteredIVT==0)

### invert the masked array to first compute total spacetime volume of non-background states, i.e. EWE states 

In [16]:
wherestates = filteredIVT == 1

In [17]:
maskedIVT = np.ma.masked_array(filteredIVT, filteredIVT==0)

In [18]:
np.array_equal(np.invert(maskedIVT.mask), wherestates)

True

total spacetime volume:

In [19]:
np.sum(wherestates) / np.size(precip_field) * 100

19.610698332609957

### total precip co-occurring with EWE states

In [20]:
maskedprecip = np.ma.masked_array(precip_field, maskedIVT.mask)

In [21]:
(np.sum(maskedprecip) / np.sum(precip_field)) * 100

48.243916034698486

## compute co-occurring precip extremes for different percentiles

In [22]:
sorted_precip = np.sort(precip_field.flatten())

In [23]:
percentile = 0.9

In [24]:
percentile * np.size(sorted_precip)

398131200.0

In [25]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [26]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [27]:
total_extremes = np.sum(precip_field > extreme_ind)

make sure percentile is working properly

In [29]:
(total_extremes / np.size(precip_field)) *100

9.999999773943866

In [30]:
state_extremes = np.sum(np.logical_and(wherestates, precip_field > extreme_ind))

percent of co-occurring extremes:

In [31]:
(state_extremes / total_extremes) * 100

44.95693732270276

In [32]:
percentile = 0.99

In [33]:
percentile * np.size(sorted_precip)

437944320.0

In [34]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [35]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [36]:
total_extremes = np.sum(precip_field > extreme_ind)

make sure percentile is working properly

In [37]:
(total_extremes / np.size(precip_field)) *100

0.9999997739438657

In [38]:
state_extremes = np.sum(np.logical_and(wherestates, precip_field > extreme_ind))

percent of co-occurring extremes:

In [39]:
(state_extremes / total_extremes) * 100

58.07559725739594

In [40]:
percentile = 0.999

In [41]:
percentile * np.size(sorted_precip)

441925632.0

In [42]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [43]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [44]:
total_extremes = np.sum(precip_field > extreme_ind)

make sure percentile is working properly

In [45]:
(total_extremes / np.size(precip_field)) *100

0.09999977394386574

In [46]:
state_extremes = np.sum(np.logical_and(wherestates, precip_field > extreme_ind))

percent of co-occurring extremes:

In [47]:
(state_extremes / total_extremes) * 100

75.2694030070055

## Calculations for zonal extremes

### tropics

In [48]:
np.shape(precip_field)

(500, 768, 1152)

In [49]:
tropic_precip = precip_field[:, stropic_lat:ntropic_lat, :]
np.shape(tropic_precip)

(500, 199, 1152)

In [50]:
tropic_states = state_field[:, stropic_lat:ntropic_lat, :]
np.shape(tropic_states)

(500, 199, 1152)

In [51]:
filteredIVT = np.ones(np.shape(tropic_states), dtype=int)
filteredIVT[tropic_states==13] = 0
filteredIVT[tropic_states==0] = 0

In [52]:
wherestates = filteredIVT == 1

In [53]:
maskedIVT = np.ma.masked_array(filteredIVT, filteredIVT==0)

In [54]:
np.sum(wherestates) / np.size(tropic_precip) * 100

38.41553514098269

In [55]:
maskedprecip = np.ma.masked_array(tropic_precip, maskedIVT.mask)

In [56]:
(np.sum(maskedprecip) / np.sum(tropic_precip)) * 100

61.93549633026123

In [57]:
sorted_precip = np.sort(tropic_precip.flatten())

In [58]:
percentile = 0.9

In [59]:
percentile * np.size(sorted_precip)

103161600.0

In [60]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [61]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [62]:
total_extremes = np.sum(tropic_precip > extreme_ind)

In [63]:
(total_extremes / np.size(tropic_precip)) *100

9.999999127582356

In [64]:
state_extremes = np.sum(np.logical_and(wherestates, tropic_precip > extreme_ind))

In [65]:
(state_extremes / total_extremes) * 100

61.733830762652744

In [66]:
percentile = 0.99

In [67]:
percentile * np.size(sorted_precip)

113477760.0

In [68]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [69]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [70]:
total_extremes = np.sum(tropic_precip > extreme_ind)

In [71]:
(total_extremes / np.size(tropic_precip)) *100

0.9999991275823562

In [72]:
state_extremes = np.sum(np.logical_and(wherestates, tropic_precip > extreme_ind))

In [73]:
(state_extremes / total_extremes) * 100

69.53384067371638

In [74]:
percentile = 0.999

In [75]:
percentile * np.size(sorted_precip)

114509376.0

In [76]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [77]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [78]:
total_extremes = np.sum(tropic_precip > extreme_ind)

In [79]:
(total_extremes / np.size(tropic_precip)) *100

0.09999912758235624

In [80]:
state_extremes = np.sum(np.logical_and(wherestates, tropic_precip > extreme_ind))

In [81]:
(state_extremes / total_extremes) * 100

76.67745565898642

### north temperate

In [82]:
np.shape(precip_field)

(500, 768, 1152)

In [83]:
north_precip = precip_field[:, ntropic_lat:arctic_lat, :]
np.shape(north_precip)

(500, 185, 1152)

In [84]:
north_states = state_field[:, ntropic_lat:arctic_lat, :]
np.shape(north_states)

(500, 185, 1152)

In [85]:
filteredIVT = np.ones(np.shape(north_states), dtype=int)
filteredIVT[north_states==13] = 0
filteredIVT[north_states==0] = 0

In [86]:
wherestates = filteredIVT == 1

In [87]:
maskedIVT = np.ma.masked_array(filteredIVT, filteredIVT==0)

In [88]:
np.sum(wherestates) / np.size(north_precip) * 100

17.92901839339339

In [89]:
maskedprecip = np.ma.masked_array(north_precip, maskedIVT.mask)

In [90]:
(np.sum(maskedprecip) / np.sum(north_precip)) * 100

46.997418999671936

In [91]:
sorted_precip = np.sort(north_precip.flatten())

In [92]:
percentile = 0.9

In [93]:
percentile * np.size(sorted_precip)

95904000.0

In [94]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [95]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [96]:
total_extremes = np.sum(north_precip > extreme_ind)

In [97]:
(total_extremes / np.size(north_precip)) *100

9.999998123123124

In [98]:
state_extremes = np.sum(np.logical_and(wherestates, north_precip > extreme_ind))

In [99]:
(state_extremes / total_extremes) * 100

39.51222588442678

In [100]:
percentile = 0.99

In [101]:
percentile * np.size(sorted_precip)

105494400.0

In [102]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [103]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [104]:
total_extremes = np.sum(north_precip > extreme_ind)

In [105]:
(total_extremes / np.size(north_precip)) *100

0.9999990615615616

In [106]:
state_extremes = np.sum(np.logical_and(wherestates, north_precip > extreme_ind))

In [107]:
(state_extremes / total_extremes) * 100

54.76713097516045

In [108]:
percentile = 0.999

In [109]:
percentile * np.size(sorted_precip)

106453440.0

In [110]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [111]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [112]:
total_extremes = np.sum(north_precip > extreme_ind)

In [113]:
(total_extremes / np.size(north_precip)) *100

0.09999906156156156

In [114]:
state_extremes = np.sum(np.logical_and(wherestates, north_precip > extreme_ind))

In [115]:
(state_extremes / total_extremes) * 100

79.21433196632852

### south temperate

In [116]:
np.shape(precip_field)

(500, 768, 1152)

In [117]:
south_precip = precip_field[:, antarctic_lat:stropic_lat, :]
np.shape(south_precip)

(500, 185, 1152)

In [118]:
south_states = state_field[:, antarctic_lat:stropic_lat, :]
np.shape(south_states)

(500, 185, 1152)

In [119]:
filteredIVT = np.ones(np.shape(south_states), dtype=int)
filteredIVT[south_states==13] = 0
filteredIVT[south_states==0] = 0

In [120]:
wherestates = filteredIVT == 1

In [121]:
maskedIVT = np.ma.masked_array(filteredIVT, filteredIVT==0)

In [122]:
np.sum(wherestates) / np.size(south_precip) * 100

20.87145082582583

In [123]:
maskedprecip = np.ma.masked_array(south_precip, maskedIVT.mask)

In [124]:
(np.sum(maskedprecip) / np.sum(south_precip)) * 100

45.299723744392395

In [125]:
sorted_precip = np.sort(south_precip.flatten())

In [126]:
percentile = 0.9

In [127]:
percentile * np.size(sorted_precip)

95904000.0

In [128]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [129]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [130]:
total_extremes = np.sum(south_precip > extreme_ind)

In [131]:
(total_extremes / np.size(south_precip)) *100

9.999998123123124

In [132]:
state_extremes = np.sum(np.logical_and(wherestates, south_precip > extreme_ind))

In [133]:
(state_extremes / total_extremes) * 100

42.107477873025125

In [134]:
percentile = 0.99

In [135]:
percentile * np.size(sorted_precip)

105494400.0

In [136]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [137]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [138]:
total_extremes = np.sum(south_precip > extreme_ind)

In [139]:
(total_extremes / np.size(south_precip)) *100

0.9999990615615616

In [140]:
state_extremes = np.sum(np.logical_and(wherestates, south_precip > extreme_ind))

In [141]:
(state_extremes / total_extremes) * 100

60.836111895750655

In [142]:
percentile = 0.999

In [143]:
percentile * np.size(sorted_precip)

106453440.0

In [144]:
percentile_ind = int(percentile * np.size(sorted_precip))

In [145]:
extreme_ind = (sorted_precip[percentile_ind] + sorted_precip[percentile_ind+1]) / 2.0

In [146]:
total_extremes = np.sum(south_precip > extreme_ind)

In [147]:
(total_extremes / np.size(south_precip)) *100

0.09999906156156156

In [148]:
state_extremes = np.sum(np.logical_and(wherestates, south_precip > extreme_ind))

In [149]:
(state_extremes / total_extremes) * 100

76.43465122608133